In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
import xgboost as xgb
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler

In [2]:
df_encode = pd.read_csv('data/loan_cleaned_v1.csv')

In [3]:
df_encode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466285 entries, 0 to 466284
Data columns (total 29 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       466285 non-null  int64  
 1   loan_amnt                466285 non-null  int64  
 2   funded_amnt              466285 non-null  int64  
 3   funded_amnt_inv          466285 non-null  float64
 4   term                     466285 non-null  object 
 5   int_rate                 466285 non-null  float64
 6   installment              466285 non-null  float64
 7   grade                    466285 non-null  object 
 8   emp_length               466285 non-null  int64  
 9   home_ownership           466285 non-null  object 
 10  annual_inc               466285 non-null  float64
 11  loan_status              466285 non-null  int64  
 12  pymnt_plan               466285 non-null  object 
 13  purpose                  466285 non-null  object 
 14  dti 

In [4]:
df_encode['term'] = df_encode['term'].apply(lambda term: int(term[:3]))

In [5]:
# Encoding / labelling data
# Replace label with same characteristic 
target_dict = {'MORTGAGE':'MORTGAGE',
               'RENT':'RENT',
               'OWN':'OWN',
               'OTHER':'OTHER',
               'ANY':'OTHER',
               'NONE':'OTHER'}

df_encode["home_ownership"] = df_encode["home_ownership"].map(target_dict)

df_encode['term'] = np.where(df_encode['term']==36,0,1)

df_encode['initial_list_status'] = np.where(df_encode['initial_list_status']=='f',0,1) 
# One Hot Encoding Categoric
for cat in [['home_ownership','purpose','grade']]:
    onehots = pd.get_dummies(df_encode[cat], prefix = cat)

In [6]:
num = df_encode.select_dtypes(include='number').columns
cat = df_encode.select_dtypes(include='object').columns

In [7]:
manual_bin = ['last_pymnt_amnt','revol_util','revol_bal','pymnt_time','term','loan_status','annual_inc']
auto_bin = num.drop(manual_bin)

feat_manual_bin = df_encode[manual_bin]
feat_auto_bin = df_encode[auto_bin]

In [8]:
# make a function
def make_bins(df, feature, cut):
    df[feature] = pd.cut(df[feature],cut)
    return df

In [9]:
# loan amnt
loan_amnt = make_bins(feat_auto_bin, 'loan_amnt',10)
loan_amnt_dum = pd.get_dummies(loan_amnt['loan_amnt'], prefix='loan_amnt')

# int_rate
int_rate = make_bins(feat_auto_bin, 'int_rate',10)
int_rate_dum = pd.get_dummies(int_rate['int_rate'], prefix='int_rate')

# dti
dti = make_bins(feat_auto_bin, 'dti', 10)
dti_dum = pd.get_dummies(dti['dti'], prefix='dti')

# open_acc
open_acc = make_bins(feat_auto_bin,'open_acc',10)
open_acc_dum = pd.get_dummies(open_acc['open_acc'], prefix='open_acc')

# total_acc
total_acc = make_bins(feat_auto_bin, 'total_acc', 7)
total_acc_dum = pd.get_dummies(total_acc['total_acc'], prefix='total_acc')

# total_pymnt
total_pymnt = make_bins(feat_auto_bin, 'total_pymnt', 10)
total_pymnt_dum = pd.get_dummies(total_pymnt['total_pymnt'], prefix='total_pymnt')

# credit_pull_yea
credit_pull_year = make_bins(feat_auto_bin,'credit_pull_year',10)
credit_pull_year_dum = pd.get_dummies(credit_pull_year['credit_pull_year'], prefix='credit_pull_year')

C:\Users\ACER\AppData\Local\Temp\ipykernel_18188\3351152094.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature] = pd.cut(df[feature],cut)
C:\Users\ACER\AppData\Local\Temp\ipykernel_18188\3351152094.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature] = pd.cut(df[feature],cut)
C:\Users\ACER\AppData\Local\Temp\ipykernel_18188\3351152094.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [10]:
num_auto_bin = pd.concat([loan_amnt_dum,int_rate_dum,dti_dum,open_acc_dum,
                     total_acc_dum,total_pymnt_dum,credit_pull_year_dum],axis=1)

In [11]:
# revol_bal
feat_manual_bin['revol_bal_(0, 5000)'] = np.where((feat_manual_bin['revol_bal']>=0)&(feat_manual_bin['revol_bal']<=5000),1,0)
feat_manual_bin['revol_bal_(5000, 10000)'] =   np.where((feat_manual_bin['revol_bal']>5000)&(feat_manual_bin['revol_bal']<=10000),1,0)
feat_manual_bin['revol_bal_(10000, 15000)'] = np.where((feat_manual_bin['revol_bal']>10000)&(feat_manual_bin['revol_bal']<=15000),1,0)
feat_manual_bin['revol_bal_(> 15000)'] = np.where(feat_manual_bin['revol_bal']>15000,1,0)

# revol_util
feat_manual_bin['revol_util_(0, 20)'] = np.where((feat_manual_bin['revol_util']>=0)&(feat_manual_bin['revol_util']<=20),1,0)
feat_manual_bin['revol_util_(20, 40)'] = np.where((feat_manual_bin['revol_util']>20)&(feat_manual_bin['revol_util']<=40),1,0)
feat_manual_bin['revol_util_(40, 60)'] = np.where((feat_manual_bin['revol_util']>40)&(feat_manual_bin['revol_util']<=60),1,0)
feat_manual_bin['revol_util_(60, 80)'] = np.where((feat_manual_bin['revol_util']>60)&(feat_manual_bin['revol_util']<=80),1,0)
feat_manual_bin['revol_util_(> 80)'] = np.where(feat_manual_bin['revol_util']>80,1,0)

# last_pymnt_amnt
feat_manual_bin['last_pymnt_amnt_(0,500)'] = np.where((feat_manual_bin['last_pymnt_amnt']>=0)&(feat_manual_bin['last_pymnt_amnt']<=500),1,0)
feat_manual_bin['last_pymnt_amnt_(500, 1000)'] = np.where((feat_manual_bin['last_pymnt_amnt']>500)&(feat_manual_bin['last_pymnt_amnt']<=1000),1,0)
feat_manual_bin['last_pymnt_amnt_(1000, 1500)'] = np.where((feat_manual_bin['last_pymnt_amnt']>1000)&(feat_manual_bin['last_pymnt_amnt']<=1500),1,0)
feat_manual_bin['last_pymnt_amnt_(1500, 3500)'] = np.where((feat_manual_bin['last_pymnt_amnt']>1500)&(feat_manual_bin['last_pymnt_amnt']<=3500),1,0)
feat_manual_bin['last_pymnt_amnt_(> 3500)'] = np.where((feat_manual_bin['last_pymnt_amnt']>3500),1,0)

# pymnt_time
feat_manual_bin['pymnt_time_(0, 1)'] = np.where((feat_manual_bin['pymnt_time']>=0)&(feat_manual_bin['pymnt_time']<=1),1,0)
feat_manual_bin['pymnt_time_(1, 6)'] = np.where((feat_manual_bin['pymnt_time']>1)&(feat_manual_bin['pymnt_time']<=6),1,0)
feat_manual_bin['pymnt_time_(6, 12)'] = np.where((feat_manual_bin['pymnt_time']>6)&(feat_manual_bin['pymnt_time']<=12),1,0)
feat_manual_bin['pymnt_time_(> 12)'] = np.where((feat_manual_bin['pymnt_time']>12),1,0)

# annual_inc
feat_manual_bin['annual_inc_(low income)'] = np.where((feat_manual_bin['annual_inc']>=0)&(feat_manual_bin['annual_inc']<=50000),1,0)
feat_manual_bin['annual_inc_(mid income)'] = np.where((feat_manual_bin['annual_inc']>50000)&(feat_manual_bin['annual_inc']<=200000),1,0)
feat_manual_bin['annual_inc_(high income)'] = np.where((feat_manual_bin['annual_inc']>200000),1,0)

#Drop orginial Feature
feat_manual_bin = feat_manual_bin.drop(manual_bin, axis=1)

C:\Users\ACER\AppData\Local\Temp\ipykernel_18188\4119999650.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feat_manual_bin['revol_bal_(0, 5000)'] = np.where((feat_manual_bin['revol_bal']>=0)&(feat_manual_bin['revol_bal']<=5000),1,0)
C:\Users\ACER\AppData\Local\Temp\ipykernel_18188\4119999650.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feat_manual_bin['revol_bal_(5000, 10000)'] =   np.where((feat_manual_bin['revol_bal']>5000)&(feat_manual_bin['revol_bal']<=10000),1,0)
C:\Users\ACER\AppData\Local

In [12]:
df_encoded = pd.concat([onehots,num_auto_bin,feat_manual_bin,df_encode['term'],df_encode['initial_list_status'],df_encode['loan_status']],axis = 1)
df_encoded.head()

,home_ownership_MORTGAGE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,...,"pymnt_time_(0, 1)","pymnt_time_(1, 6)","pymnt_time_(6, 12)",pymnt_time_(> 12),annual_inc_(low income),annual_inc_(mid income),annual_inc_(high income),term,initial_list_status,loan_status
0,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
1,0,0,0,1,1,0,0,0,0,0,...,0,0,0,1,1,0,0,1,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
4,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,1


In [13]:
# Save clean data to csv
path_file = 'data/loan_encode.csv'
df_encoded.to_csv(path_file, index=False)